### This Notebook will be mainly used for the Capstone Project for the IBM Data Science Professional Certificate. The project involves application of Data Science techniques for segmenting and clustring of neighborhoods.

In [2]:
#Importing pandas and numpy libraries 
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


# This part of this Notebook will be used for exploring, segmenting and clustering the neighbourhoods within the City of Toronto, Canada.

### The first step is to dowload all dependencies and libraries required to achieve the task

In [1]:
#Importing numpy to handle data in vectorised manner or format
import numpy as np

#Importing the pandas dataframe for data analysis
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Importing json to read json files
import json

#Installing geopy to enable the geocoding library which converts address to latitude and longitude
!conda install -c conda-forge geopy --yes

#Importing Nominatim to convert address into latitude and longitude
from geopy.geocoders import Nominatim

#Importing a library to handle requests
#import requests
#Importing the required Matplotlib libraries
import matplotlib.cm as cm
import matplotlib.colors as colors
#Importing K-Means algorithm from sklearn for clustering
from sklearn.cluster import KMeans
#Installing folium library for map rendering
!conda install -c conda-forge folium=0.5.0 --yes
#Importing the installed folium library
print('Libraries installed!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [2]:
#Importing the json_normalise function to transform json file into a pandas dataframe
from pandas.io.json import json_normalize
print('Json normalize imported')

Json normalize imported


In [3]:
#Importing a library to handle requests
import requests

In [4]:
#Requesting the data from wikipedia and defining the url
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
#Importing Beautiful Soup Framework to enable website data scraping
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_url, 'html.parser')# parsing html and not xml
print(soup)

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xc6wNgpAICEAABZJe4EAAACR","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":926306543,"wgRevisionId":926306543,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontari

#### After analysing the html data extracted, it can be observed that all the Toronto Neighbourhoods are under table class 'wikitable sortable'

In [7]:
#Finding all attributes of wikitable sortable
table = soup.find('table',{'class':'wikitable sortable'})
print (table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [8]:
#Defining empty lists for each column of the table to be populated later
pst_code = [] # Post code
bro = [] # borough
neighb = [] # neighbourhood

#Populating the table as an array row by row using nested for loops
# 'tr' tag is for each row within the table
# 'td' tag is for each cell within the table
for row in table.findAll('tr'): 
    cells = row.findAll('td') 
    if len(cells)==3: # sanity check
        pst_code.append(cells[0].find(text = True))
        bro.append(cells[1].find(text = True))
        neighb.append(cells[2].find(text = True))        

In [9]:
#Now converting the Post code,borough, Neighbourhood lists into a dataframe
df = pd.DataFrame()
df['Post Code'] = pst_code
df['Borough'] = bro
df['Neighbourhood'] = neighb
df.head()


,Post Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
#Processing cells with only Borough assigned while ignoring all cells without Borough assigned.
df2 = df
wantd_rows = df2[df2['Borough'] != "Not assigned"]
wantd_rows.head()

,Post Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [12]:
#Replacing the "Not assigned" Neighbourhood with its assigned Borough
df3 = wantd_rows
df3['Neighbourhood'].replace('Not assigned',"Queen's Park", inplace = True)
df3.head()

,Post Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [13]:
#Checking for duplicate post codes but for different neighbourhoods
df3.groupby('Post Code')
df3.head()

,Post Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [14]:
#Combining Neighbourhoods with the same Post Code into one row but neighbourhood names comma separated
df4 = df3.groupby(['Post Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df4.columns = ['Post Code', 'Borough', 'Neighbourhood']
df4.head()


,Post Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n


In [15]:
#Printing the final rows in the data frame.
df4.shape

(103, 3)

### The next step is to extract latitude and longitude data for each postcode and convert the data into a dataframe. 

In [16]:
#Reading the csv file with geolocation data for each postcode
df_geodata = pd.read_csv('https://cocl.us/Geospatial_data')
print('Geolocation data loaded!')

Geolocation data loaded!


In [17]:
#Displaying a few rows of the geo dataframe
df_geodata.columns = ['Post Code','Latitude', 'Longitude']
df_geodata.head()

,Post Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### The next step is to merge the geodata dataframe with the first data frame with neighbourhoods data.  This creates a single dataframe which will be used with the Foursquare map for segmenting and clustering neighbourhoods.

In [18]:
#Merging the two dataframes ON the ['Post Code'] column which is common in both dataframes
df_merg = pd.merge(df4, df_geodata, on = ['Post Code'], how = 'inner')
df_merg.head()

,Post Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae\n,43.773136,-79.239476


In [19]:
#Sanity check to see if the rows are still the same number as in each individual dataframe before merger
df_merg.shape

(103, 5)

### The next step is to use geopy to obtain the latitude and longitude values of Toronto
##### The geocoder user_agent shall be called toronto_explorer

In [23]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

Solving environment: done

# All requested packages already installed.



In [24]:
#Installaing and importing folium, the map rendering library
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium
print ('Folium imported!')

Folium imported!


In [25]:
#Obtaining the values of latitude and longitude for Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto City are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto City are 43.653963, -79.387207.


### The next step is to create the map of Toronto with its neighbourhoods imposed on top

In [26]:
#Creating the map of Toronto using folium and the values of latitude and longitude
toronto_map = folium.Map( location = (latitude, longitude), zoom_start = 10)

#Adding circle markers to the map using for loop
tor_neighbs = df_merg
for lati, longi, borough, neighbourhood in zip (tor_neighbs['Latitude'], tor_neighbs['Longitude'], tor_neighbs['Borough'], tor_neighbs['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lati, longi],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(toronto_map)
toronto_map

## Now we are going to focus on exploring and clustering neighbourhoods in the borough of Central Toronto

### The first step is to slice the tor_neighbs dafaframe and create a new dataframe for the Central Toronto Borough

In [27]:
cent_toronto = tor_neighbs[tor_neighbs['Borough'] == 'Central Toronto'].reset_index(drop = True)
cent_toronto.head()

,Post Code,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North\n,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West\n,43.715383,-79.405678
3,M4S,Central Toronto,Davisville\n,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East\n",43.689574,-79.383160


### Next step is to obtain the geo coordinates for Central Toronto

In [28]:
#Obtaining the values of latitude and longitude for Central Toronto
address = 'Central Toronto, ON'

geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates for Central Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates for Central Toronto are 43.653963, -79.387207.


In [29]:
#Creating the map of Central Toronto borough using folium and the values of latitude and longitude
central_toronto_map = folium.Map( location = (latitude, longitude), zoom_start = 12)

#Adding circle markers to the map using for loop
for lati, longi, borough, neighbourhood in zip (cent_toronto['Latitude'], cent_toronto['Longitude'], cent_toronto['Borough'], cent_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lati, longi],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(central_toronto_map)
central_toronto_map

### Next step is to use the Foursquare API to explore and segment the neighbourhoods within Central Toronto borough

In [31]:
# The code was removed by Watson Studio for sharing.

### The next step is to explore each neighbourhood in the Borough of Central Toronto. This will be achieved by using a function to carry out the following procedure:

1. Extracting each neighbourhood's geographical coordinates.
2. Extracting the top 100 venues (including category type) within a 500m radius for each neighbourhood.
3. Clean the json data and convert the data into a pandas dataframe.

In [32]:
#Defining the to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Creating the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Making the GET request to extract json data
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            longi, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
#Calling the getNearbyVenues function for each neighbourhood to create a dataframe called cent_toronto_venues
LIMIT = 100
cent_toronto_venues = getNearbyVenues (names = cent_toronto['Neighbourhood'],
                                       latitudes = cent_toronto['Latitude'],
                                       longitudes = cent_toronto['Longitude']
                                      )

Lawrence Park
Davisville North

North Toronto West

Davisville

Moore Park, Summerhill East

Deer Park, Forest Hill SE
, Rathnelly, South Hill, Summerhill West

Roselawn

Forest Hill North, Forest Hill West

The Annex, North Midtown
, Yorkville


In [35]:
#Size of resulting dataframe
cent_toronto_venues.shape

(113, 7)

In [36]:
#First 5 rows of the resulting cent_toronto_venues dataframe
cent_toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.405678,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.405678,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.405678,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North\n,43.712751,-79.405678,Sherwood Park,43.716551,-79.387776,Park
4,Davisville North\n,43.712751,-79.405678,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


 ### Checking how many venues have been returned for each neighbourhood and how many venue categories can be curated from the returned data.

In [37]:
#Number of venues for each neighbourhood
cent_toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Davisville\n,34,34,34,34,34,34
Davisville North\n,9,9,9,9,9,9
"Deer Park, Forest Hill SE\n, Rathnelly, South Hill, Summerhill West\n",15,15,15,15,15,15
"Forest Hill North, Forest Hill West\n",5,5,5,5,5,5
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East\n",3,3,3,3,3,3
North Toronto West\n,20,20,20,20,20,20
Roselawn\n,2,2,2,2,2,2
"The Annex, North Midtown\n, Yorkville",22,22,22,22,22,22


In [38]:
#Checking unique venue categories
print('There are {} uniques categories.'.format(len(cent_toronto_venues['Venue Category'].unique())))

There are 57 uniques categories.


### Next step is to analyze each neighbourhood

In [39]:
# Central Toronto one hot encoding
central_toronto_onehot = pd.get_dummies(cent_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_toronto_onehot['Neighbourhood'] = cent_toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [central_toronto_onehot.columns[-1]] + list(central_toronto_onehot.columns[:-1])
central_toronto_onehot = central_toronto_onehot[fixed_columns]

central_toronto_onehot.head()

,Neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Dessert Shop,Diner,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North\n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North\n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
central_toronto_onehot.shape

(113, 58)

### The next step is to group rows by neighbourhood and by taking the mean of the frequemcy of occurence of each venue category

In [41]:
#Remember neighbourhoods are grouped by the same Post Code.
cent_toronto_groupd = central_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
cent_toronto_groupd

,Neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Dessert Shop,Diner,Farmers Market,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville\n,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.058824,0.00,0.000000,0.058824,0.000000,0.000000,0.029412,0.088235,0.029412,0.029412,0.029412,0.000000,0.029412,0.0,0.00,0.029412,0.029412,0.00,0.058824,0.000000,0.0,0.000000,0.029412,0.058824,0.0,0.000000,0.000000,0.00,0.029412,0.029412,0.058824,0.000000,0.000000,0.00,0.029412,0.00,0.088235,0.029412,0.00,0.0,0.000000,0.000000,0.058824,0.000000,0.058824,0.029412,0.0,0.000000,0.000000,0.00
1,Davisville North\n,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.111111,0.000000,0.0,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.111111,0.000000,0.111111,0.000000,0.000000,0.00,0.000000,0.00,0.111111,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
2,"Deer Park, Forest Hill SE\n, Rathnelly, South ...",0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.066667,0.066667,0.00,0.000000,0.000000,0.066667,0.000000,0.133333,0.00,0.066667,0.00,0.000000,0.000000,0.00,0.0,0.066667,0.066667,0.066667,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,0.00
3,"Forest Hill North, Forest Hill West\n",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00,0.200000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00
4,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.00
5,"Moore Park, Summerhill East\n",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.333333,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.333333,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00
6,North Toronto West\n,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.050000,0.05,0.100000,0.100000,0.000000,0.000000,0.000000,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.0,0.05,0.000000,0.000000,0.05,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0

In [42]:
#checking shape of the resulting dataframe
cent_toronto_groupd.shape

(9, 58)

### Next step is to print each neighbourhood with its top 6 most common venues

In [43]:
#Predefining the number of top venues to print. This number can be changed to suit.
top_venues = 6

# Using for loop to achieve the requirement. Remember that neighbourhoods are grouped by the same Post Code.
for neighbourhood in cent_toronto_groupd['Neighbourhood']:
    print("----" + neighbourhood+ "----")
    temp = cent_toronto_groupd[cent_toronto_groupd['Neighbourhood'] == neighbourhood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop=True).head(top_venues))
    print('\n')
    

----Davisville
----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2                Café  0.06
3  Italian Restaurant  0.06
4    Sushi Restaurant  0.06
5         Coffee Shop  0.06


----Davisville North
----
               venue  freq
0              Hotel  0.11
1                Gym  0.11
2  Convenience Store  0.11
3     Clothing Store  0.11
4     Sandwich Place  0.11
5        Pizza Place  0.11


----Deer Park, Forest Hill SE
, Rathnelly, South Hill, Summerhill West
----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2  American Restaurant  0.07
3           Restaurant  0.07
4          Pizza Place  0.07
5           Sports Bar  0.07


----Forest Hill North, Forest Hill West
----
                 venue  freq
0        Jewelry Store   0.2
1                Trail   0.2
2             Bus Line   0.2
3                 Park   0.2
4     Sushi Restaurant   0.2
5  American Restaurant   0.0


----Lawrence Park----
            

### The next step is put the results into a pandas dataframe

First sub-step is to define a function which sorts the top venues in descending order 

In [44]:
#This function returns most frequent or most common venues
def return_top_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

The next sub-step is to create the pandas dataframe and display top 7 venues in each neighbourhood. Remember each neighbourhood is grouped by the same Post Code.

In [46]:
#Predefining the number of top venues. This number can be changed if we need to print a different number of top venues e.g 9
top_venues = 7

indicators = ['st', 'nd', 'rd']

#Creating number of columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#Create the new dataframe
neighbs_venues_sorted = pd.DataFrame(columns=columns)
neighbs_venues_sorted['Neighbourhood'] = cent_toronto_groupd['Neighbourhood']

for ind in np.arange(cent_toronto_groupd.shape[0]):
    neighbs_venues_sorted.iloc[ind, 1:] = return_top_venues(cent_toronto_groupd.iloc[ind, :], top_venues)

neighbs_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Davisville\n,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Gym,Sushi Restaurant
1,Davisville North\n,Hotel,Clothing Store,Gym,Food & Drink Shop,Park,Pizza Place,Sandwich Place
2,"Deer Park, Forest Hill SE\n, Rathnelly, South ...",Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store
3,"Forest Hill North, Forest Hill West\n",Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Fried Chicken Joint,Diner
4,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Diner,Discount Store,Farmers Market


## Next step is to cluster the neighbourhoods using k-Means Clustering Algorithm

The neighbourhoods are going to be clustered into 4 clusters

In [47]:
#Setting the number of clusters. This can be changed depending on number of clusters required
k = 4

cent_toronto_groupd_cluster = cent_toronto_groupd.drop('Neighbourhood', 1)

#Running k-means clustering
kmeans_cluster = KMeans(n_clusters=k, random_state=0).fit(cent_toronto_groupd_cluster)

#Checking cluster labels generated for each row in the dataframe
kmeans_cluster.labels_[0:10] 

array([2, 2, 2, 0, 3, 0, 2, 1, 2], dtype=int32)

Creating a dataframe that includes the kmeans_cluster and top 7 venues for each neighbourhood

In [48]:
#Adding clustering labels
neighbs_venues_sorted.insert(0, 'Cluster Labels', kmeans_cluster.labels_)

cent_toronto_merged = cent_toronto

#Merging toronto_grouped with central toronto data (cent_toronto) to add latitude and longitude for each neighborhood
cent_toronto_merged = cent_toronto_merged.join(neighbs_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

cent_toronto_merged.head()

,Post Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Bus Line,Park,Swim School,Yoga Studio,Diner,Discount Store,Farmers Market
1,M4P,Central Toronto,Davisville North\n,43.712751,-79.390197,2,Hotel,Clothing Store,Gym,Food & Drink Shop,Park,Pizza Place,Sandwich Place
2,M4R,Central Toronto,North Toronto West\n,43.715383,-79.405678,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Shoe Store,Health & Beauty Service,Diner,Dessert Shop
3,M4S,Central Toronto,Davisville\n,43.704324,-79.388790,2,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Gym,Sushi Restaurant
4,M4T,Central Toronto,"Moore Park, Summerhill East\n",43.689574,-79.383160,0,Trail,Park,Playground,Yoga Studio,Deli / Bodega,Gym,Greek Restaurant


### Next step is to show the clusters on a map

In [57]:
#Using folium to create the map
cent_toronto_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

#Setting colour scheme for the clusters. k is the number of clusters defined above
x = np.arange(k)
y = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(y)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#Adding markers superimposed onto the map
markers_colors = []
for lat, lon, poi, cluster in zip(cent_toronto_merged['Latitude'], cent_toronto_merged['Longitude'], cent_toronto_merged['Neighbourhood'], cent_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(cent_toronto_map_clusters)
#Displaying the clusters on a map       
cent_toronto_map_clusters

## Next step is to examine the clusters to examine the disnguishing features for each cluster

## Cluster 1

In [53]:
cent_toronto_merged.loc[cent_toronto_merged['Cluster Labels'] == 0, cent_toronto_merged.columns[[1] + list(range(5, cent_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
4,Central Toronto,0,Trail,Park,Playground,Yoga Studio,Deli / Bodega,Gym,Greek Restaurant
7,Central Toronto,0,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Fried Chicken Joint,Diner


## Cluster 2

In [54]:
cent_toronto_merged.loc[cent_toronto_merged['Cluster Labels'] == 1, cent_toronto_merged.columns[[1] + list(range(5, cent_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
6,Central Toronto,1,Ice Cream Shop,Garden,Pool,Yoga Studio,History Museum,Gym / Fitness Center,Gym


In [55]:
cent_toronto_merged.loc[cent_toronto_merged['Cluster Labels'] == 2, cent_toronto_merged.columns[[1] + list(range(5, cent_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
1,Central Toronto,2,Hotel,Clothing Store,Gym,Food & Drink Shop,Park,Pizza Place,Sandwich Place
2,Central Toronto,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Shoe Store,Health & Beauty Service,Diner,Dessert Shop
3,Central Toronto,2,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Thai Restaurant,Gym,Sushi Restaurant
5,Central Toronto,2,Pub,Coffee Shop,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Liquor Store
8,Central Toronto,2,Café,Sandwich Place,Coffee Shop,American Restaurant,History Museum,Furniture / Home Store,Indian Restaurant


In [56]:
cent_toronto_merged.loc[cent_toronto_merged['Cluster Labels'] == 3, cent_toronto_merged.columns[[1] + list(range(5, cent_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Central Toronto,3,Bus Line,Park,Swim School,Yoga Studio,Diner,Discount Store,Farmers Market
